In [29]:
 #!/usr/bin/env python3

import gradio as gr
import os
from os.path import join, dirname
from dotenv import load_dotenv
from dotenv import load_dotenv
import conversational
# from conversational import conversatioalChatGPT

# from conversation import create_conversation
# dotenv_path = join(dirname(__file__), '.env')
# print(load_dotenv(dotenv_path))

import os

from dotenv import load_dotenv
from abc import ABCMeta, abstractmethod


from langchain.embeddings import OpenAIEmbeddings
import indexdb
from langchain.chains import (
    ConversationalRetrievalChain,
    LLMChain
)

from langchain import HuggingFaceHub
# from utilities import load_api_key


# from langchain.vectorstores import Chroma
# from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOpenAI , ChatAnthropic
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

# from langchain.chains.chat_vector_db.prompts import (CONDENSE_QUESTION_PROMPT,
#                                                      QA_PROMPT)
"""Create a ChatVectorDBChain for question/answering."""
# from langchain.callbacks.base import AsyncCallbackManager
from langchain.callbacks.manager import AsyncCallbackManager
# from langchain.callbacks.tracers import LangChainTracer
from langchain.chains import ChatVectorDBChain
from langchain.chains.llm import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.vectorstores.base import VectorStore
from langchain.callbacks.tracers import LangChainTracer
import  openai

from transformers import TFAutoModelForQuestionAnswering



In [30]:
from conversational import conversatioalBase

class conversatioalChatGPT(conversatioalBase):
    
    def __init__(self,embedding=None,retriver=None,chat_prompt_template=os.environ.get("CONDENSE_TEMPLATE"),qa_template=os.environ.get("QA_TEMPLATE")) -> None:
        super().__init__(embedding,retriver)
        self.chat_prompt_template=chat_prompt_template
        self.qa_template=qa_template
        
    
    def getDefaultEmbadding(self): 
        return OpenAIEmbeddings(
            openai_api_key=os.environ.get("OPENAI_API_KEY")
        )
        
    
    
    def getConversational(self)-> ConversationalRetrievalChain:

    
    

        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False
        )
    
        # def getretriver(self):
        #     # return dbCroma
        #     # return Faissindexdb(embedding=self.getDefaultEmbadding())
        #     return self.retriver
    

        CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(self.chat_prompt_template)
        QA_PROMPT= PromptTemplate.from_template(self.qa_template)
        
        # define two LLM models from OpenAI
        question_gen_llm = ChatOpenAI(
            temperature=0.2,
            # max_tokens=2000,
            # model_name="text-curie-001" ,#text-davinci-003",
        )
        
        
        
        return  ConversationalRetrievalChain.from_llm(
            llm=question_gen_llm,
            # chain_type='stuff',
            chain_type="map_reduce",
            retriever=self.retriver.getDb().as_retriever(qa_template=QA_PROMPT, question_generator_template=CONDENSE_QUESTION_PROMPT),
            memory=memory,
            get_chat_history=lambda h: h,
            # verbose=True,
            # return_source_documents=True
        )

In [51]:

qa=conversatioalChatGPT().getConversational()

In [52]:
# from indexdb import Faissindexdb
import indexdb
query="איך אני לוקך משכנתה?"
fidb=indexdb.Faissindexdb()
print(fidb.getDb().similarity_search(query))
docs = fidb.getDb().similarity_search(query)


TypeError: Faissindexdb.__init__() missing 1 required positional argument: 'embedding'

In [17]:
res = qa(
        {
            'question': query,
            'chat_history': []
        }
    )
print( res )

{'question': 'איך אני לוקך משכנתה?', 'chat_history': 'Human: מיסים\nAI: נקודות זיכוי ממס ניתנות לפי מצב משפחתי, גיל, מין, השכלה, שירות צבאי ועוד. לכל נקודת זיכוי יש שווי כספי (הנקבע בכל שנה), כשנקודות הזיכוי מקטינות את הסכום שממנו מחושב המס.\nHuman: מיסים\nAI: There is no relevant text in the given portion of the document to answer this question.', 'answer': 'To calculate your mortgage, you should take into account the total cost of the property and your own equity. The maximum funding ratio that a buyer can receive from the bank for a mortgage is determined by the Bank of Israel. For a single apartment, the maximum funding ratio is 75%, which means that your own equity should be at least 25% of the property value. For a duplex apartment, the maximum funding ratio is 70%, which means that your own equity should be at least 30% of the property value. For an investment property, the maximum funding ratio is 50%, which means that your own equity should be at least 50% of the property valu